In [ ]:
!pip install konlpy

In [ ]:
import torch.nn as nn
from transformers import BertModel

class KoBERTIntentSlotModel(nn.Module):
    def __init__(self, num_intents, num_slots):
        super().__init__()
        self.bert = BertModel.from_pretrained("skt/kobert-base-v1")
        hidden_size = self.bert.config.hidden_size

        self.intent_classifier = nn.Linear(hidden_size, num_intents)
        self.slot_classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, num_slots)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        pooled_output = outputs.pooler_output

        intent_logits = self.intent_classifier(pooled_output)
        slot_logits = self.slot_classifier(sequence_output)

        return intent_logits, slot_logits


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from torch.optim import AdamW
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from collections import Counter
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
import torch.nn.functional as F
import ast
import json
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

tqdm.pandas()

# 📊 데이터 로드 및 전처리
def load_and_preprocess_data():
    df = pd.read_csv("intent_slot_dataset_cleaned.csv")

    df["intent_list"] = df["intent_list"].apply(
        lambda x: json.loads(x) if isinstance(x, str) and x.strip().startswith("[") else (x if isinstance(x, list) else [])
    )

    return df

def convert_to_bio_word_level(sentence, slot_json):
    tokens = sentence.split()
    labels = ['O'] * len(tokens)

    for slot_name, slot_value in slot_json.items():
        slot_values = slot_value if isinstance(slot_value, list) else [slot_value]
        for val in slot_values:
            if not isinstance(val, str):
                continue
            val_tokens = val.split()

            # ✅ 수정: 모든 매칭을 찾아서 처리
            i = 0
            while i <= len(tokens) - len(val_tokens):
                if tokens[i:i+len(val_tokens)] == val_tokens:
                    # 이미 태깅된 부분이 아닐 때만 태깅
                    if all(labels[i+k] == 'O' for k in range(len(val_tokens))):
                        labels[i] = f'B-{slot_name}'
                        for j in range(1, len(val_tokens)):
                            labels[i + j] = f'I-{slot_name}'
                    i += len(val_tokens)  # 매칭된 부분 다음부터 계속 찾기
                else:
                    i += 1

    return tokens, labels

def create_bio_dataset(df_combined):
    bio_data = []
    for idx, (_, row) in enumerate(df_combined.iterrows()):
        sentence = str(row['question']).strip()
        if not sentence or sentence.lower() == 'nan':
            continue

        raw_slots = row['slots']
        try:
            if isinstance(raw_slots, str):
                if raw_slots.strip() in ['', '[]', '{}', 'nan']:
                    slot_dict = {}
                else:
                    cleaned = raw_slots.replace('null','None').replace('true','True').replace('false','False')
                    slot_dict = ast.literal_eval(cleaned)
            elif isinstance(raw_slots, dict):
                slot_dict = raw_slots
            else:
                slot_dict = {}

            tokens, labels = convert_to_bio_word_level(sentence, slot_dict)
            bio_data.append({
                "intent_list": row.get("intent_list", []),  # ✅ 여기만 유지
                "tokens": tokens,
                "labels": labels
            })
        except Exception:
            continue
    return pd.DataFrame(bio_data)

def create_mappings(df_bio):
      """Intent 및 Slot 매핑 생성 - Multi-label 지원"""
      df_bio['labels'] = df_bio['labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

      # Multi-label intent 처리: 모든 개별 intent를 수집
      all_intents = set()
      for intent_list in df_bio['intent_list']:
          if isinstance(intent_list, list):
              all_intents.update(intent_list)
          else:
              all_intents.add(intent_list)

      # Intent 매핑 (개별 intent 기준)
      intent_labels = sorted(all_intents)
      intent2idx = {label: i for i, label in enumerate(intent_labels)}
      idx2intent = {i: label for label, i in intent2idx.items()}

      # Slot 라벨 수집 및 B/I 확장
      original_labels = set()
      for label_list in df_bio['labels']:
          for label in label_list:
              original_labels.add(label)
              if label.startswith('B-'):
                  original_labels.add('I-' + label[2:])

      # Slot 매핑
      slot_labels = sorted(original_labels)
      slot2idx = {label: i for i, label in enumerate(slot_labels)}
      idx2slot = {i: label for label, i in slot2idx.items()}

      return intent2idx, idx2intent, slot2idx, idx2slot

def create_balanced_dataset(df_bio, target_intent_samples=1155, target_slot_samples=300):
    """데이터 밸런싱 - Multi-label (intent_list) 지원"""

    # ✅ stratify/그룹핑용 임시 키: intent_list를 정렬 튜플로
    intent_tuple_key = df_bio['intent_list'].apply(lambda x: tuple(sorted(x)) if isinstance(x, list) else (str(x),))

    # ⬇️ intent 조합별로 업/다운샘플
    balanced_intent_dfs = []
    for combo in intent_tuple_key.unique():
        mask = intent_tuple_key == combo
        intent_df = df_bio[mask]

        if len(intent_df) >= target_intent_samples:
            sampled_df = resample(intent_df, replace=False, n_samples=target_intent_samples, random_state=42)
        else:
            sampled_df = resample(intent_df, replace=True,  n_samples=target_intent_samples, random_state=42)

        balanced_intent_dfs.append(sampled_df)

    df_balanced_intent = pd.concat(balanced_intent_dfs, ignore_index=True)

    # ⬇️ 슬롯 라벨 분포 보정
    slot_counter = Counter(label for labels in df_balanced_intent['labels'] for label in labels)
    rare_slots = [label for label, cnt in slot_counter.items() if cnt < target_slot_samples]

    slot_augmented_dfs = []
    for rare_label in rare_slots:
        slot_df = df_bio[df_bio['labels'].apply(lambda lst: rare_label in lst)]
        if len(slot_df) == 0:
            continue
        needed = target_slot_samples - slot_counter[rare_label]
        if needed > 0:
            dup_df = resample(slot_df, replace=True, n_samples=needed, random_state=42)
            slot_augmented_dfs.append(dup_df)

    df_balanced = pd.concat([df_balanced_intent] + slot_augmented_dfs, ignore_index=True) if slot_augmented_dfs else df_balanced_intent

    # ✅ 디버그용 출력(의도 조합 분포)
    counts = intent_tuple_key.value_counts()
    print("✅ Intent 조합 개수:", len(counts))
    print("상위 10개 조합:")
    for combo, cnt in counts.head(10).items():
        print(f"  {combo}: {cnt}")

    print(f"\n✅ 최종 데이터셋 크기: {len(df_balanced)}")
    return df_balanced

# 🏷️ BCEWithLogitsLoss 지원 Dataset 클래스
class IntentSlotDataset(Dataset):
    def __init__(self, encodings, slot_labels, intents, intent2idx, use_bce=True):
        self.encodings = encodings
        self.slot_labels = slot_labels
        self.intents = intents
        self.intent2idx = intent2idx
        self.use_bce = use_bce
        self.num_intents = len(intent2idx)

    def __len__(self):
        return len(self.intents)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.slot_labels[idx])

        # Multi-label intent를 one-hot vector로 변환
        intent_vector = torch.zeros(self.num_intents)
        intent_labels = self.intents[idx]

        if isinstance(intent_labels, list):
            for intent_idx in intent_labels:
                intent_vector[intent_idx] = 1
        else:
            intent_vector[intent_labels] = 1

        item['intent'] = intent_vector
        return item

def align_labels_with_tokenizer(tokens, labels, tokenizer):
    """토큰과 라벨 정렬"""
    bert_tokens = []
    aligned_labels = []

    for token, label in zip(tokens, labels):
        sub_tokens = tokenizer.tokenize(token)
        if not sub_tokens:
            sub_tokens = [tokenizer.unk_token]

        bert_tokens.extend(sub_tokens)
        aligned_labels.append(label)

        for _ in range(1, len(sub_tokens)):
            if label == "O":
                aligned_labels.append("O")
            elif label.startswith("B-"):
                aligned_labels.append("I-" + label[2:])
            elif label.startswith("I-"):
                aligned_labels.append(label)
            else:
                aligned_labels.append("O")

    return bert_tokens, aligned_labels

def _ensure_list(x):
    # intent_list가 리스트/JSON/문자열 어떤 형태여도 리스트로 표준화
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        s = x.strip()
        if s.startswith('[') and s.endswith(']'):
            try:
                arr = json.loads(s)
                return [str(v).strip() for v in arr if str(v).strip()]
            except Exception:
                pass
        return [t.strip() for t in s.split(',') if t.strip()]
    return [] if pd.isna(x) else [str(x).strip()]

def encode_data(df, tokenizer, intent2idx, slot2idx, max_len=64):
    """데이터 인코딩 - Multi-label (intent_list) 지원"""
    input_ids, attention_masks, slot_label_ids, intent_ids = [], [], [], []

    for _, row in df.iterrows():
        tokens = row["tokens"]
        labels = row["labels"]

        # 토크나이저 정렬
        bert_tokens, aligned_labels = align_labels_with_tokenizer(tokens, labels, tokenizer)
        label_ids = [slot2idx[label] for label in aligned_labels]

        # 길이 자르기
        if len(bert_tokens) > max_len - 2:
            bert_tokens = bert_tokens[:max_len - 2]
            label_ids   = label_ids[:max_len - 2]

        # [CLS], [SEP] / 패딩
        tokens_input = ['[CLS]'] + bert_tokens + ['[SEP]']
        label_ids    = [slot2idx['O']] + label_ids + [slot2idx['O']]

        input_id       = tokenizer.convert_tokens_to_ids(tokens_input)
        attention_mask = [1] * len(input_id)

        pad_len = max_len - len(input_id)
        if pad_len > 0:
            input_id       += [0] * pad_len
            attention_mask += [0] * pad_len
            label_ids      += [slot2idx['O']] * pad_len

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        slot_label_ids.append(label_ids)

        # ✅ Multi-label intent_list → index 리스트
        intents = _ensure_list(row.get("intent_list", []))
        intent_indices = [intent2idx[i] for i in intents if i in intent2idx]

        # 최소 1개는 보장(없으면 빈 리스트 유지해도 됨; Dataset에서 원핫 만들 때 0으로만 채워짐)
        intent_ids.append(intent_indices)

    encodings = {"input_ids": input_ids, "attention_mask": attention_masks}
    return encodings, slot_label_ids, intent_ids

def create_datasets(df_balanced, tokenizer, intent2idx, slot2idx, use_bce=True, max_len=64, test_size=0.1):
      """Dataset 및 DataLoader 생성 - Stratify 문제 해결"""

      df_balanced['intent_tuple'] = df_balanced['intent_list'].apply(lambda x: tuple(sorted(x)))

      # Train/Val 분할
      train_df, val_df = train_test_split(
          df_balanced,
          test_size=test_size,
          stratify=df_balanced['intent_tuple'],
          random_state=42
      )

      # intent_str 컬럼 제거
      train_df = train_df.drop(columns=['intent_tuple'])
      val_df = val_df.drop(columns=['intent_tuple'])

      # 인코딩
      train_encodings, train_slot_labels, train_intents = encode_data(train_df, tokenizer, intent2idx, slot2idx,
  max_len)
      val_encodings, val_slot_labels, val_intents = encode_data(val_df, tokenizer, intent2idx, slot2idx, max_len)

      # Dataset 생성
      train_dataset = IntentSlotDataset(train_encodings, train_slot_labels, train_intents, intent2idx, use_bce)
      val_dataset = IntentSlotDataset(val_encodings, val_slot_labels, val_intents, intent2idx, use_bce)

      # DataLoader 생성
      train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
      val_loader = DataLoader(val_dataset, batch_size=16)

      print(f"✅ {'BCE' if use_bce else 'CrossEntropy'}용 데이터셋 생성 완료")
      print(f"   Train: {len(train_dataset)} samples")
      print(f"   Val: {len(val_dataset)} samples")

      return train_loader, val_loader

# 🔧 학습 및 평가 함수
def train_epoch_bce(model, dataloader, optimizer, device, intent2idx, slot2idx, slot_weights=None):
    """BCE 손실을 사용한 학습"""
    model.train()
    total_loss = 0

    # Loss functions
    intent_loss_fn = BCEWithLogitsLoss()
    if slot_weights is not None:
        slot_loss_fn = nn.CrossEntropyLoss(weight=slot_weights)
    else:
        slot_loss_fn = CrossEntropyLoss()

    for batch in tqdm(dataloader, desc="🛠️ Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        slot_labels = batch['labels'].to(device)
        intent_labels = batch['intent'].to(device)

        optimizer.zero_grad()
        intent_logits, slot_logits = model(input_ids, attention_mask)

        # Intent loss (BCE)
        loss_intent = intent_loss_fn(intent_logits, intent_labels)

        # Slot loss (CrossEntropy)
        loss_slot = slot_loss_fn(slot_logits.view(-1, len(slot2idx)), slot_labels.view(-1))

        loss = loss_intent + loss_slot

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)

def evaluate_bce(model, dataloader, device, intent2idx, slot2idx, threshold=0.5):
    """BCE 기반 평가"""
    model.eval()
    intent_preds, intent_trues = [], []
    slot_preds, slot_trues = [], []

    idx2intent = {v: k for k, v in intent2idx.items()}
    idx2slot = {v: k for k, v in slot2idx.items()}

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="🔍 Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            slot_labels = batch['labels'].to(device)
            intent_labels = batch['intent'].to(device)

            intent_logits, slot_logits = model(input_ids, attention_mask)

            # Intent 예측 (sigmoid + threshold)
            intent_probs = torch.sigmoid(intent_logits)
            intent_pred = (intent_probs > threshold).float()

            intent_preds.extend(intent_pred.cpu().numpy())
            intent_trues.extend(intent_labels.cpu().numpy())

            # Slot 예측
            slot_pred = torch.argmax(slot_logits, dim=2)
            for i in range(slot_labels.size(0)):
                true_seq = slot_labels[i].cpu().tolist()
                pred_seq = slot_pred[i].cpu().tolist()
                for t, p in zip(true_seq, pred_seq):
                    if t != -100:
                        slot_trues.append(t)
                        slot_preds.append(p)

    # Intent 정확도 계산
    intent_preds = np.array(intent_preds)
    intent_trues = np.array(intent_trues)

    exact_match = np.all(intent_preds == intent_trues, axis=1)
    intent_acc = np.mean(exact_match)

    intent_report = classification_report(
        intent_trues, intent_preds,
        target_names=list(intent2idx.keys()),
        zero_division=0
    )

    # Slot 정확도 계산
    support_counter = Counter(slot_trues)
    nonzero_labels = [i for i in slot2idx.values() if support_counter[i] > 0]
    target_names_nonzero = [key for key, val in slot2idx.items() if val in nonzero_labels]

    slot_acc = accuracy_score(slot_trues, slot_preds)
    slot_report = classification_report(
        slot_trues, slot_preds,
        labels=nonzero_labels,
        target_names=target_names_nonzero,
        zero_division=0
    )

    return intent_acc, intent_report, slot_acc, slot_report

def train_with_bce(model, train_loader, val_loader, device, intent2idx, slot2idx,
                   slot_weights=None, epochs=10, lr=5e-5, threshold=0.5, save_path=None):
    """BCEWithLogitsLoss로 학습"""

    optimizer = AdamW(model.parameters(), lr=lr)

    # 결과 저장용
    train_losses = []
    val_intent_accuracies = []
    val_slot_accuracies = []

    best_val_intent_acc = 0
    best_model_state = None
    best_intent_report = ""
    best_slot_report = ""

    print("🚀 BCEWithLogitsLoss Training Started")
    print(f"📊 Threshold: {threshold}")
    print("=" * 50)

    for epoch in range(epochs):
        print(f"\n📚 Epoch {epoch+1}/{epochs}")

        # 학습
        train_loss = train_epoch_bce(
            model, train_loader, optimizer, device,
            intent2idx, slot2idx, slot_weights
        )

        # 평가
        val_intent_acc, intent_report, val_slot_acc, slot_report = evaluate_bce(
            model, val_loader, device, intent2idx, slot2idx, threshold
        )

        # 결과 저장
        train_losses.append(train_loss)
        val_intent_accuracies.append(val_intent_acc)
        val_slot_accuracies.append(val_slot_acc)

        print(f"📉 Train Loss: {train_loss:.4f}")
        print(f"🎯 Val Intent Accuracy: {val_intent_acc:.4f}")
        print(f"🏷 Val Slot Accuracy: {val_slot_acc:.4f}")

        # Best model 저장
        if val_intent_acc > best_val_intent_acc:
            best_val_intent_acc = val_intent_acc
            best_model_state = model.state_dict().copy()
            best_intent_report = intent_report
            best_slot_report = slot_report
            print("✅ Best model updated!")

            # 모델 저장
            if save_path:
                import os
                import pickle
                os.makedirs(save_path, exist_ok=True)
                torch.save(best_model_state, os.path.join(save_path, "best_model.pt"))
                with open(os.path.join(save_path, "intent2idx.pkl"), "wb") as f:
                    pickle.dump(intent2idx, f)
                with open(os.path.join(save_path, "slot2idx.pkl"), "wb") as f:
                    pickle.dump(slot2idx, f)

    print(f"\n🎉 Training Completed!")
    print(f"📈 Best Intent Accuracy: {best_val_intent_acc:.4f}")
    print("\n📊 Best Intent Classification Report:")
    print(best_intent_report)
    print("\n📊 Best Slot Classification Report:")
    print(best_slot_report)

    return {
        'best_model_state': best_model_state,
        'train_losses': train_losses,
        'val_intent_accuracies': val_intent_accuracies,
        'val_slot_accuracies': val_slot_accuracies,
        'best_intent_acc': best_val_intent_acc,
        'best_intent_report': best_intent_report,
        'best_slot_report': best_slot_report
    }

def plot_training_curves(results, save_path=None):
    """학습 곡선 시각화"""
    train_losses = results['train_losses']
    val_intent_accuracies = results['val_intent_accuracies']
    val_slot_accuracies = results['val_slot_accuracies']

    epochs = list(range(1, len(train_losses) + 1))

    fig, ax1 = plt.subplots(figsize=(10, 5))

    # 왼쪽 Y축: Train Loss
    ax1.plot(epochs, train_losses, color='blue', label='Train Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # 오른쪽 Y축: Accuracy
    ax2 = ax1.twinx()
    ax2.plot(epochs, val_intent_accuracies, color='orange', label='Intent Accuracy')
    ax2.plot(epochs, val_slot_accuracies, color='green', label='Slot Accuracy')
    ax2.set_ylabel('Accuracy', color='green')
    ax2.tick_params(axis='y', labelcolor='green')

    # 범례
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='lower center')

    plt.title('📊 BCEWithLogitsLoss Training Results')
    plt.grid(True)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"📊 Plot saved to: {save_path}")

    plt.show()

# 🚀 전체 파이프라인 실행 함수
def run_bce_training_pipeline(model_class, save_path="best_models/intent-bce-v1"):
    """전체 BCEWithLogitsLoss 학습 파이프라인"""

    print("🔄 1단계: 데이터 로드 및 전처리...")
    df_combined = load_and_preprocess_data()
    df_bio = create_bio_dataset(df_combined)

    print("🔄 2단계: 매핑 생성...")
    intent2idx, idx2intent, slot2idx, idx2slot = create_mappings(df_bio)

    print("🔄 3단계: 데이터 밸런싱...")
    df_balanced = create_balanced_dataset(df_bio)

    print("🔄 4단계: 토크나이저 로드...")
    tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)

    print("🔄 5단계: 데이터셋 생성...")
    train_loader, val_loader = create_datasets(
        df_balanced, tokenizer, intent2idx, slot2idx, use_bce=True
    )

    print("🔄 6단계: 모델 초기화...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model_class(len(intent2idx), len(slot2idx)).to(device)

    print("🔄 7단계: 학습 시작...")
    results = train_with_bce(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        intent2idx=intent2idx,
        slot2idx=slot2idx,
        epochs=5,
        threshold=0.5,
        save_path=save_path
    )

    print("🔄 8단계: 결과 시각화...")
    plot_training_curves(results, f"{save_path}/training_curves.png" if save_path else None)

    return results, intent2idx, slot2idx

# 💡 사용 예시
if __name__ == "__main__":
    print("🎯 Unified BCEWithLogitsLoss Training Pipeline")
    print("=" * 60)

    results, intent2idx, slot2idx = run_bce_training_pipeline(KoBERTIntentSlotModel)

    print("""
    사용법:
    1. 모델 클래스 import: from your_model import KoBERTIntentSlotModel
    2. 파이프라인 실행: results, intent2idx, slot2idx = run_bce_training_pipeline(KoBERTIntentSlotModel)

    또는 단계별 실행:
    1. df_combined = load_and_preprocess_data()
    2. df_bio = create_bio_dataset(df_combined)
    3. intent2idx, idx2intent, slot2idx, idx2slot = create_mappings(df_bio)
    4. df_balanced = create_balanced_dataset(df_bio)
    5. tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)
    6. train_loader, val_loader = create_datasets(df_balanced, tokenizer, intent2idx, slot2idx)
    7. results = train_with_bce(model, train_loader, val_loader, device, intent2idx, slot2idx)
    """)

In [ ]:
 AIRLINE_CODES = [
    "KE", "OZ", "7C", "LJ", "BX", "ZE", "TW", "RS", "YP", "RF", "KJ", "2I", "Q5",
    "FE", "AQ", "Y6", "GB", "K4", "YJ", "GP", "3V", "5O", "PH", "4A", "CJ", "A0",
    "B5", "8H", "H5", "2C", "OK", "DX", "L3", "ES", "D0", "Q7", "D5", "7T", "LI",
    "8D", "C0", "9A", "IV", "A5", "II", "AZ", "XE", "FK", "KL", "WA", "KK", "LV",
    "GE", "LO", "WW", "7M", "M2", "OM", "MB", "6V", "MU", "2P", "PB", "CG", "OH",
    "S7", "RZ", "5S", "SK", "SL", "SP", "O3", "KI", "S0", "8F", "DT", "T0", "TP",
    "TP", "YQ", "RO", "X3", "OR", "6B", "TB", "BY", "GO", "5X", "UD", "BS", "UJ",
    "RT", "V6", "2Z", "YG", "GA", "GT", "G2", "IF", "GF", "GW", "G8", "JS", "G7",
    "6G", "G3", "Y5", "G1", "5U", "GX", "CN", "YR", "GV", "HB", "Q9", "Z5", "G6",
    "5S", "GE", "ON", "NP", "NJ", "T2", "NM", "IN", "SA", "N8", "9Y", "NA", "NE",
    "NO", "RA", "1I", "NC", "Y7", "N4", "ND", "N7", "Z0", "N0", "DY", "DY", "D8",
    "NA", "O9", "NG", "VQ", "0N", "HW", "J3", "DD", "N5", "BJ", "EJ", "7H", "OJ",
    "9J", "DN", "D3", "DL", "R6", "ZQ", "DZ", "KB", "3R", "4Y", "DF", "LC", "B0",
    "R4", "TH", "LK", "QV", "LW", "7S", "FR", "RK", "JT", "8V", "M3", "UC", "L7",
    "LA", "JJ", "XL", "LA", "4C", "PZ", "LP", "LQ", "TM", "P7", "BN", "8L", "WZ",
    "L5", "7H", "QL", "R0", "LC", "ZL", "LM", "FV", "AT", "BI", "RW", "RJ", "3Q",
    "RG", "L8", "L9", "7R", "5R", "LH", "CL", "LH", "VL", "LG", "NJ", "GJ", "LT",
    "WB", "YL", "LN", "5U", "4P", "EX", "8N", "F2", "YX", "L2", "L0", "RI", "FC",
    "UJ", "DI", "O8", "MP", "4M", "M7", "MY", "2M", "2Y", "N7", "C6", "LL", "5G",
    "MJ", "9T", "W5", "6M", "AE", "7Y", "NR", "3W", "DB", "MH", "VM", "DJ", "4X",
    "N5", "BM", "YV", "WD", "L6", "Q2", "MT", "XF", "ME", "M4", "UB", "8M", "K7",
    "J4", "OD", "ID", "UP", "PG", "V9", "VK", "QH", "NB", "2T", "J8", "RR", "VS",
    "VA", "XR", "BD", "J0", "8E", "CH", "3B", "JV", "JD", "VN", "0V", "V4", "B2",
    "AB", "VI", "N3", "Q6", "Y4", "V7", "OB", "2L", "U4", "RP", "UZ", "VY", "B3",
    "4B", "LB", "FB", "TF", "NT", "TT", "1X", "SN", "MX", "SI", "BV", "BZ", "BO",
    "BZ", "BG", "VB", "BH", "UK", "V4", "VJ", "VU", "B4", "UQ", "NT", "VP", "OL",
    "OL", "SV", "S1", "WN", "2S", "9R", "F2", "ZF", "FA", "SC", "OV", "SO", "FM",
    "MF", "SH", "CE", "IH", "9X", "PL", "SY", "R8", "SR", "2U", "S6", "2R", "YH",
    "EZ", "WG", "ZH", "DK", "ER", "D2", "5J", "DG", "C2", "K3", "PV", "IS", "9M",
    "5Z", "SZ", "6J", "FW", "IE", "SP", "SK", "DR", "SD", "SJ", "PY", "Y8", "N9",
    "IU", "UL", "5N", "6Y", "2N", "QS", "6D", "3Z", "7O", "2E", "LX", "WT", "Y3",
    "ML", "3E", "BQ", "ZA", "GQ", "H2", "RD", "H8", "UY", "S8", "U3", "QU", "GG",
    "BC", "PQ", "U5", "OW", "OO", "LC", "M8", "TE", "QN", "DO", "DV", "TR", "HK",
    "S5", "2I", "JX", "4E", "4R", "7G", "SG", "SG", "P8", "NK", "C7", "YR", "5G",
    "RB", "BB", "SI", "A2", "QG", "WX", "K5", "3M", "US", "7L", "ZP", "7E", "SQ",
    "SQ", "XQ", "3U", "XO", "EH", "DM", "AG", "6A", "JI", "AR", "FG", "W3", "Z8",
    "MZ", "A8", "M6", "AA", "8R", "XP", "YK", "4B", "ZR", "GU", "A0", "2K", "TA",
    "WC", "QT", "LR", "AV", "9V", "X9", "X8", "JU", "4K", "KP", "0A", "8V", "P9",
    "KP", "GM", "AM", "6R", "5D", "E4", "SU", "FW", "FI", "Q7", "F7", "2O", "I4",
    "J2", "S4", "ZF", "AD", "2F", "AJ", "QP", "AK", "V8", "5Y", "RC", "J7", "AW",
    "XU", "BU", "8U", "6L", "O4", "KO", "J5", "AS", "JN", "G4", "DQ", "6R", "KH",
    "VC", "QQ", "UJ", "AP", "2B", "G0", "5A", "2G", "Q3", "A2", "IZ", "YE", "YC",
    "R3", "AN", "4W", "9I", "UE", "A3", "WK", "B8", "EA", "ET", "EK", "BR", "8K",
    "5V", "GD", "ES", "GT", "GL", "4N", "PX", "NZ", "EN", "HD", "RM", "GZ", "7I",
    "LZ", "ZM", "MD", "NX", "KM", "MV", "MK", "4O", "NF", "KF", "BP", "2J", "C7",
    "PJ", "HC", "JU", "HM", "Y2", "GI", "PF", "G9", "3O", "3L", "E5", "KC", "2A",
    "NY", "ZB", "6I", "AH", "TZ", "CC", "UU", "3N", "N6", "ZW", "X5", "UX", "3H",
    "AI", "IX", "9H", "CA", "CA", "DJ", "UM", "3C", "3E", "2Q", "TX", "TY", "AC",
    "RV", "AC", "KS", "XK", "HF", "YN", "IK", "VT", "TN", "ST", "TC", "TS", "A6",
    "8C", "6C", "8T", "7P", "FS", "AF", "F4", "P4", "HT", "LD", "AO", "TL", "KA",
    "8G", "N2", "NL", "3S", "M0", "V5", "KW", "2S", "7L", "F5", "ZV", "XZ", "K2",
    "EI", "EI", "4Z", "X8", "BT", "4Y", "RU", "PA", "T6", "AK", "D7", "ED", "NQ",
    "JK", "EF", "SM", "SB", "P2", "RE", "ZD", "VF", "8J", "ZU", "ET", "EY", "EE",
    "9E", "MQ", "E4", "G4", "7Q", "EL", "LY", "BS", "BA", "IY", "YT", "O7", "HZ",
    "6O", "GR", "OC", "OG", "UI", "WY", "OF", "OS", "YI", "BK", "OA", "OY", "EB",
    "WP", "Q9", "R5", "UR", "U6", "UQ", "HY", "PS", "4W", "OX", "WL", "3G", "2W",
    "3P", "KD", "WU", "PN", "WS", "WR", "WC", "WF", "W6", "W4", "5W", "W9", "7W",
    "WH", "P5", "IW", "UA", "B7", "U7", "6U", "QY", "H6", "PS", "YU", "Q4", "E6",
    "EW", "YZ", "UF", "UT", "RF", "X7", "H7", "IA", "EP", "B9", "IR", "IO", "E9",
    "I2", "YW", "IB", "QI", "E7", "T3", "I8", "MG", "RD", "7Z", "U2", "DS", "EC",
    "MS", "EO", "E7", "XN", "7A", "QE", "QZ", "6E", "I7", "JY", "I4", "8B", "V8",
    "IJ", "JL", "JC", "KZ", "JO", "J9", "JM", "ZN", "RY", "JG", "QK", "NU", "LC",
    "NH", "JL", "ZK", "B6", "WJ", "JA", "JZ", "3K", "GK", "JQ", "4J", "LS", "JO",
    "JR", "A9", "GH", "6J", "3J", "IM", "HO", "CZ", "CZ", "CO", "MU", "G5", "I9",
    "CK", "CI", "XM", "D4", "ZG", "9D", "TZ", "KN", "CF", "VC", "6Q", "X7", "5C",
    "GS", "HT", "EU", "GM", "9C", "OQ", "QW", "C3", "6L", "CV", "C8", "W8", "PM",
    "HH", "8F", "NV", "V3", "BW", "QC", "Z7", "9Q", "IQ", "QR", "VR", "A7", "K4",
    "K9", "MO", "RQ", "K6", "KR", "5T", "AU", "CX", "C5", "6C", "LF", "GY", "HQ",
    "KQ", "QB", "8K", "KX", "9K", "M5", "4K", "CD", "XC", "XR", "SS", "KO", "GW",
    "CQ", "7C", "CM", "P5", "FC", "DE", "V0", "C4", "8Z", "QF", "CU", "KU", "KY",
    "QO", "QA", "KV", "C8", "OU", "VE", "KG", "FK", "Z3", "TB", "3T", "SF", "SL",
    "VZ", "FD", "XJ", "TG", "IT", "ZT", "TM", "HJ", "K3", "TK", "TI", "TQ", "BV",
    "T5", "T9", "U8", "TU", "UG", "M8", "T7", "8B", "HV", "TO", "R2", "Q8", "T7",
    "C3", "TJ", "9N", "IL", "TD", "T7", "TV", "5P", "ZP", "P6", "FY", "PK", "HP",
    "OP", "8Y", "P1", "8P", "BL", "PC", "7V", "FX", "FN", "UF", "DP", "PD", "NI",
    "PO", "PI", "Z4", "FU", "6P", "BF", "P0", "F9", "FS", "FA", "4F", "E3", "3X",
    "MI", "FH", "P6", "PW", "SX", "WV", "XY", "FL", "5M", "IF", "S9", "G6", "EQ",
    "8W", "F0", "5F", "FT", "9P", "FP", "FZ", "FO", "FA", "F3", "3F", "F6", "YS",
    "D3", "PU", "F8", "OG", "W2", "PT", "FJ", "MM", "PE", "AY", "HI", "Z2", "PR",
    "YB", "HA", "5K", "3L", "HU", "HG", "H4", "H7", "HR", "NS", "HN", "H3", "H3",
    "JB", "2L", "QX", "5Q", "HD", "UO", "HX", "RH", "RS", "WI", "JH", "MR", "HJ",
    "H9", "OI",

    "HL" # 학습데이터에 있는 가짜 항공편 코드
]

In [ ]:
import re
from konlpy.tag import Okt

okt = Okt()

def clean_text(text):
    """
    KoBERT 기반 전처리에 적합하도록 특수문자 제거 및 공백 정리
    """
    # 한글, 영문, 숫자, 공백만 남기기
    text = re.sub(r"[^\uAC00-\uD7A3a-zA-Z0-9\s]", "", str(text))
    # 다중 공백 제거
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# 플레이스홀더
FLIGHT_PREFIX = "FLIGHT"   # 토큰은 ⟪FLIGHT0⟫, ⟪FLIGHT1⟫ ... 형태로 생성
TERMINAL_PREFIX = "TERMINAL"  # 토큰은 ⟪TERMINAL0⟫, ⟪TERMINAL1⟫ ... 형태로 생성
# 유효한 항공사 코드만 매칭하는 패턴 생성
airline_codes_pattern = '|'.join(AIRLINE_CODES)

# 일반적인 항공편 패턴 (공백 없음)
flight_pattern_normal = re.compile(rf'\b({airline_codes_pattern})\s*[-]?\s*(\d{{1,4}})\b', re.IGNORECASE)

# 띄어쓰기된 항공편 패턴 (예: "HL 7201", "7 C 0102")
flight_pattern_spaced = re.compile(r'\b([A-Za-z0-9])\s+([A-Za-z0-9])\s+(\d{1,4})\b', re.IGNORECASE)

def _collapse_flight_spans(text: str) -> str:
    """항공편 표현을 항상 붙여쓰기(하이픈/공백 제거) + 대문자로 통일."""
    # 일반 패턴 처리 (ke 907 -> KE907, KE 907 -> KE907)
    text = flight_pattern_normal.sub(lambda m: (m.group(1) + m.group(2)).upper(), text)

    # 띄어쓰기된 패턴 처리 (7 c 0102 -> 7C0102, hl 7201 -> HL7201)
    def spaced_replacer(m):
        code = m.group(1) + m.group(2)  # 항공사 코드 결합
        number = m.group(3)             # 항공편 번호
        # 유효한 항공사 코드인지 확인
        if code.upper() in AIRLINE_CODES:
            return (code + number).upper()  # 대문자로 변환
        return m.group(0)  # 매칭되지 않으면 원본 유지

    text = flight_pattern_spaced.sub(spaced_replacer, text)
    return text

def _collapse_terminal_spans(text: str) -> str:
    """터미널 표현을 T1, T2로 정규화."""
    # T1 관련 패턴들
    t1_patterns = [
        r'(?:제?\s*1\s*(?:여객\s*)?터미널|터미널\s*1|T\s*-?\s*1|첫\s*번?\s*째\s*(?:여객\s*)?터미널|제일\s*(?:여객\s*)?터미널)',
        r'(?:일\s*(?:여객\s*)?터미널|터미널\s*일)',
        r'(?:제\s*1\s*여객\s*터미널|제1\s*여객\s*터미널)',
    ]

    # T2 관련 패턴들
    t2_patterns = [
        r'(?:제?\s*2\s*(?:여객\s*)?터미널|터미널\s*2|T\s*-?\s*2|두\s*번?\s*째\s*(?:여객\s*)?터미널|제이\s*(?:여객\s*)?터미널)',
        r'(?:이\s*(?:여객\s*)?터미널|터미널\s*이)',
        r'(?:제\s*2\s*여객\s*터미널|제2\s*여객\s*터미널)',
    ]

    # T1으로 정규화
    for pattern in t1_patterns:
        text = re.sub(pattern, 'T1', text, flags=re.IGNORECASE)

    # T2로 정규화
    for pattern in t2_patterns:
        text = re.sub(pattern, 'T2', text, flags=re.IGNORECASE)

    return text

_FACILITY_LIST = ["기도실", "검역장", "수유실"]  # 필요하면 여기에 계속 추가
def _collapse_keyword(text: str, word: str) -> str:
    base = word.replace(" ", "")
    # 한글/영문/숫자 경계에서만 매치되게 경계 추가
    pattern = r'(?<![가-힣A-Za-z0-9])' + r'\s*'.join(map(re.escape, base)) + r'(?![가-힣A-Za-z0-9])'
    return re.sub(pattern, base, text)

def _collapse_facility_spans(text: str) -> str:
    for w in _FACILITY_LIST:
        text = _collapse_keyword(text, w)
    return text

def normalize_with_morph(text: str) -> str:
    # 0) 특수문자 제거 및 공백 정리
    processed_text = clean_text(text)

    # 1) 항공편을 먼저 붙여쓰기 정규화 (KE 907 -> KE907)
    processed_text = _collapse_flight_spans(processed_text)

    # 1.5) 터미널 표현 정규화 (1터미널 -> T1, 제2터미널 -> T2)
    processed_text = _collapse_terminal_spans(processed_text)

    # 2) 항공편을 플레이스홀더로 치환 (여러 개 지원)
    flight_map = {}  # 예: {'⟪FLIGHT0⟫': 'KE907', '⟪FLIGHT1⟫': 'VS5501'}
    flight_counter = 0
    def _flight_repl(m):
        nonlocal flight_counter
        code = (m.group(1) + m.group(2)).upper()     # 붙여쓰기 + 대문자 변환
        token = f'⟪{FLIGHT_PREFIX}{flight_counter}⟫'
        flight_map[token] = code
        flight_counter += 1
        return token

    processed_text = flight_pattern_normal.sub(_flight_repl, processed_text)

    # 2.5) 터미널을 플레이스홀더로 치환
    terminal_map = {}  # 예: {'⟪TERMINAL0⟫': 'T1', '⟪TERMINAL1⟫': 'T2'}
    terminal_counter = 0
    def _terminal_repl(m):
        nonlocal terminal_counter
        terminal_code = m.group(0)  # T1 또는 T2
        token = f'⟪{TERMINAL_PREFIX}{terminal_counter}⟫'
        terminal_map[token] = terminal_code
        terminal_counter += 1
        return token

    # T1, T2 패턴을 플레이스홀더로 치환
    terminal_pattern = re.compile(r'\bT[12]\b')
    processed_text = terminal_pattern.sub(_terminal_repl, processed_text)


    # 3) 형태소 분석 (정규화/어간화 끔)
    tokens = okt.morphs(processed_text, norm=False, stem=False)

    # 4) 다시 문자열로 합치기
    text_after = " ".join(tokens)

    # 5) 플레이스홀더 복원
    #    토크나이즈가 공백을 끼워넣어도(⟪ FLIGHT 0 ⟫) 정확히 복원되도록 처리
    for token, code in flight_map.items():
        core = token[1:-1]  # 'FLIGHT0'
        m = re.match(r'(FLIGHT)(\d+)$', core)
        if m:
            pat = re.compile(r'⟪\s*' + m.group(1) + r'\s*' + m.group(2) + r'\s*⟫')
            text_after = pat.sub(code, text_after)
        # 혹시 그대로 남아있으면 직접 치환
        text_after = text_after.replace(token, code)

    # 터미널 플레이스홀더 복원
    for token, code in terminal_map.items():
        core = token[1:-1]  # 'TERMINAL0'
        m = re.match(r'(TERMINAL)(\d+)$', core)
        if m:
            pat = re.compile(r'⟪\s*' + m.group(1) + r'\s*' + m.group(2) + r'\s*⟫')
            text_after = pat.sub(code, text_after)
        # 혹시 그대로 남아있으면 직접 치환
        text_after = text_after.replace(token, code)

    # 6) 혹시 남은 공백/하이픈 변형을 다시 한 번 정규화
    text_after = _collapse_flight_spans(text_after)
    text_after = _collapse_terminal_spans(text_after)

    text_after = _collapse_facility_spans(text_after)

    # 7) 공백 정리
    text_after = re.sub(r'\s+', ' ', text_after).strip()
    return text_after


In [ ]:
import torch
import torch.nn as nn
from torch.nn.functional import softmax, sigmoid
from transformers import BertModel, AutoTokenizer
import pickle
import numpy as np
from typing import List, Tuple, Dict

# 🔧 설정 및 경로
INTENT2IDX_PATH = "best_models/intent-bce-v1/intent2idx.pkl"
SLOT2IDX_PATH = "best_models/intent-bce-v1/slot2idx.pkl"
MODEL_PATH = "best_models/intent-bce-v1/best_model.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🏗️ 모델 클래스 (기존과 동일)
class KoBERTIntentSlotModel(nn.Module):
    def __init__(self, num_intents, num_slots):
        super().__init__()
        self.bert = BertModel.from_pretrained("skt/kobert-base-v1")
        hidden_size = self.bert.config.hidden_size

        self.intent_classifier = nn.Linear(hidden_size, num_intents)
        self.slot_classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, num_slots)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        pooled_output = outputs.pooler_output

        intent_logits = self.intent_classifier(pooled_output)
        slot_logits = self.slot_classifier(sequence_output)

        return intent_logits, slot_logits

# ✅ 인덱스 맵 및 모델 로딩
def load_model_and_mappings():
    """모델과 매핑 정보 로드"""
    # 인덱스 맵 로딩
    with open(INTENT2IDX_PATH, "rb") as f:
        intent2idx = pickle.load(f)
    with open(SLOT2IDX_PATH, "rb") as f:
        slot2idx = pickle.load(f)

    idx2intent = {v: k for k, v in intent2idx.items()}
    idx2slot = {v: k for k, v in slot2idx.items()}

    # 모델 로드
    model = KoBERTIntentSlotModel(num_intents=len(intent2idx), num_slots=len(slot2idx))
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.to(device)
    model.eval()

    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)

    return model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot

# 🧱 토큰 → 단어 병합 + 슬롯 정렬
def merge_tokens_and_slots(tokens, slot_ids, idx2slot):
    """토큰과 슬롯 병합"""
    merged = []
    word = ''
    current_slot = ''

    for token, slot_id in zip(tokens, slot_ids):
        slot = idx2slot.get(slot_id, 'O')

        if token in ['[CLS]', '[SEP]', '[PAD]']:
            continue

        if token.startswith("▁"):  # 새 단어 시작
            if word:
                merged.append((word, current_slot))
            word = token[1:]
            current_slot = slot
        else:
            word += token.replace("▁", "")

    if word:
        merged.append((word, current_slot))

    return merged

# 🔮 BCEWithLogitsLoss 기반 예측 함수
def predict_with_bce(text, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
                     threshold=0.5, top_k_intents=3, max_length=64):
    """
    BCEWithLogitsLoss로 학습된 모델을 위한 예측 함수

    Args:
        text: 입력 텍스트
        threshold: Intent 분류 임계값 (default: 0.5)
        top_k_intents: 상위 K개 인텐트 반환 (default: 3)
    """
    encoding = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=max_length
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        intent_logits, slot_logits = model(input_ids, attention_mask)

        # Intent 예측 (Sigmoid 기반)
        intent_probs = sigmoid(intent_logits)[0]  # [num_intents]

        # 임계값 이상의 인텐트들 찾기
        high_confidence_intents = []
        for i, prob in enumerate(intent_probs):
            if prob.item() >= threshold:
                intent_name = idx2intent[i]
                high_confidence_intents.append((intent_name, prob.item()))

        # 확률 순으로 정렬
        high_confidence_intents.sort(key=lambda x: x[1], reverse=True)

        # 만약 임계값 이상인 게 없다면 최고 확률 하나만
        if not high_confidence_intents:
            max_idx = torch.argmax(intent_probs).item()
            max_prob = intent_probs[max_idx].item()
            high_confidence_intents = [(idx2intent[max_idx], max_prob)]

        # Top-K 인텐트 (전체 순위용)
        topk_probs, topk_indices = torch.topk(intent_probs, min(top_k_intents, len(intent2idx)))
        all_top_intents = [(idx2intent[idx.item()], prob.item())
                          for idx, prob in zip(topk_indices, topk_probs)]

        # 슬롯 예측 (기존과 동일 - Softmax 기반)
        slot_pred_ids = torch.argmax(slot_logits, dim=2)[0].tolist()
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        merged_slots = merge_tokens_and_slots(tokens, slot_pred_ids, idx2slot)

    return {
        'high_confidence_intents': high_confidence_intents,  # 임계값 이상
        'all_top_intents': all_top_intents,                  # 전체 Top-K
        'slots': merged_slots,
        'is_multi_intent': len(high_confidence_intents) > 1,
        'max_intent_prob': max(prob for _, prob in all_top_intents),
        'intent_probs_raw': intent_probs.cpu().numpy()
    }

# 🎯 라우팅 결정 함수 (3구간 임계값)
def make_routing_decision(text, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
                         tau_hi=0.8, tau_lo=0.3, multi_threshold=0.5):
    """
    3구간 임계값 기반 라우팅 결정

    Args:
        tau_hi: 높은 임계값 (바로 라우팅)
        tau_lo: 낮은 임계값 (gray zone)
        multi_threshold: 복합 의도 판단 임계값
    """
    result = predict_with_bce(
        text, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
        threshold=multi_threshold
    )

    max_prob = result['max_intent_prob']
    is_multi = result['is_multi_intent']

    # 복합 의도인 경우
    if is_multi:
        decision = "multi_intent"
        action = f"🧠 메인 LLM 처리: 복합 의도 ({len(result['high_confidence_intents'])}개)"
        llm_type = "main"
    # 단일 의도 + 높은 신뢰도
    elif max_prob >= tau_hi:
        decision = "route"
        top_intent = result['all_top_intents'][0][0]
        action = f"✅ 직접 라우팅: {top_intent} 핸들러 호출"
        llm_type = None
    # 단일 의도 + 낮은 신뢰도
    else:
        decision = "abstain"
        action = "🧠 메인 LLM 처리: 신뢰도 낮음, 전체 의도 분석 필요"
        llm_type = "main"

    return {
        'decision': decision,
        'action': action,
        'llm_type': llm_type,
        'confidence': max_prob,
        'intents': result['high_confidence_intents'],
        'all_intents': result['all_top_intents'],
        'slots': result['slots'],
        'is_multi_intent': is_multi
    }

# 🔍 상세 분석 함수
def analyze_prediction(text, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
                      threshold=0.5, show_all_probs=False):
    """상세한 예측 분석"""
    result = predict_with_bce(
        text, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
        threshold=threshold
    )

    print(f"\n📝 입력: {text}")
    print(f"🎯 임계값: {threshold}")
    print(f"🔢 복합 의도 여부: {'Yes' if result['is_multi_intent'] else 'No'}")

    print(f"\n🏆 임계값 이상 인텐트 ({len(result['high_confidence_intents'])}개):")
    for i, (intent, prob) in enumerate(result['high_confidence_intents'], 1):
        print(f"   {i}. {intent}: {prob:.4f}")

    print(f"\n📊 전체 Top-{len(result['all_top_intents'])} 인텐트:")
    for i, (intent, prob) in enumerate(result['all_top_intents'], 1):
        print(f"   {i}. {intent}: {prob:.4f}")

    print(f"\n🎭 슬롯 태깅 결과:")
    for word, slot in result['slots']:
        print(f"   - {word}: {slot}")

    if result['is_multi_intent']:
        print(f"\n🎯 복합 의도 감지됨!")

    return result

# 🧪 인터랙티브 테스트 함수
def interactive_test():
    """인터랙티브 테스트"""
    print("🚀 BCEWithLogitsLoss 기반 인텐트/슬롯 예측기")
    print("=" * 50)

    # 모델 로드
    print("📥 모델 로딩 중...")
    model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot = load_model_and_mappings()
    print("✅ 모델 로딩 완료!")

    print(f"📊 인텐트 클래스: {len(intent2idx)}개")
    print(f"📊 슬롯 클래스: {len(slot2idx)}개")
    print("\n💡 사용법:")
    print("  - 텍스트 입력 시 예측 및 라우팅 결정 결과 표시")
    print("  - 임계값 변경: /threshold [값]")
    print("  - 종료: exit")


    threshold = 0.5 # Default threshold for analyze_prediction
    multi_threshold = 0.5 # Default threshold for make_routing_decision

    while True:
        user_input = input(f"\n✉️ 입력 (Analyze Thresh={threshold:.2f}, Multi Thresh={multi_threshold:.2f}): ").strip()

        user_input = normalize_with_morph(user_input)
        if user_input.lower() == "exit":
            print("👋 종료합니다.")
            break

        if user_input.startswith("/threshold"):
            try:
                parts = user_input.split()
                if len(parts) > 1:
                    new_threshold = float(parts[1])
                    threshold = max(0.0, min(1.0, new_threshold))
                    print(f"🎯 상세 분석 임계값 변경: {threshold:.2f}")
                if len(parts) > 2:
                    new_multi_threshold = float(parts[2])
                    multi_threshold = max(0.0, min(1.0, new_multi_threshold))
                    print(f"🎯 복합 의도 임계값 변경: {multi_threshold:.2f}")
                elif len(parts) == 2:
                    print("💡 복합 의도 임계값도 함께 변경하려면 `/threshold [분석 임계값] [복합 의도 임계값]` 형식으로 입력하세요.")

            except:
                print("❌ 사용법: /threshold [분석 임계값] [복합 의도 임계값 (선택 사항)]")
            continue

        # Process any input as a query
        if user_input:
            # Routing decision
            routing_result = make_routing_decision(
                user_input, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
                multi_threshold=multi_threshold
            )
            print(f"\n--- 라우팅 결정 ---")
            print(f"🎯 결정: {routing_result['decision'].upper()}")
            print(f"📊 최대 신뢰도: {routing_result['confidence']:.4f}")
            print(f"🔄 액션: {routing_result['action']}")
            if routing_result['intents']:
                 intents_str = ", ".join([f"{intent}({prob:.3f})"
                                          for intent, prob in routing_result['intents']])
                 print(f"🏷️ 예측 의도 (임계값 {multi_threshold:.2f} 이상): {intents_str}")

            # Detailed analysis
            print(f"\n--- 상세 예측 분석 ---")
            analyze_prediction(
                user_input, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
                threshold=threshold, show_all_probs=False # show_all_probs는 항상 False로 유지
            )


# 🎮 간단한 예측 함수 (기존 스타일 호환)
def predict_top_k_intents_and_slots(text, k=3, threshold=0.5):
    """기존 스타일과 호환되는 간단한 예측 함수"""
    model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot = load_model_and_mappings()

    result = predict_with_bce(
        text, model, tokenizer, intent2idx, idx2intent, slot2idx, idx2slot,
        threshold=threshold, top_k_intents=k
    )

    # 기존 형식으로 반환
    intents = result['all_top_intents']
    slots = result['slots']

    return intents, slots


# 🚀 메인 실행
if __name__ == "__main__":
    # 인터랙티브 모드
    interactive_test()

    # 또는 간단한 테스트
    # intents, slots = predict_top_k_intents_and_slots("내일 비행기 시간표 알려주세요")
    # print("인텐트:", intents)
    # print("슬롯:", slots)